#### 1. Download the college dataset:

In [ ]:
%env BASE_DIR=/home/jupyter/prep

In [ ]:
!gsutil cp gs://cs327e-open-access/fs_college.zip $BASE_DIR

In [ ]:
!unzip $BASE_DIR/fs_college.zip

#### 2. Create and populate Firestore database

In [ ]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('fs_college/class.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

for row in rows: 
    record = {}
    record['cno'] = row[0]
    record['cname'] = row[1]
    record['credits'] = row[2]
    
    class_ref = db.collection('class').document(row[0])
    batch.set(class_ref, record)
    
batch.commit()

In [ ]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('fs_college/student.csv', sep=',', header=0, lineterminator='\n')
student_rows = df.values.tolist()

df = pd.read_csv('fs_college/takes.csv', sep=',', header=0, lineterminator='\n')
takes_rows = df.values.tolist()

for student_row in student_rows:
    student_record = {}
    student_record['sid'] = student_row[0]
    student_record['fname'] = student_row[1]
    student_record['lname'] = student_row[2]
    student_record['dob'] = student_row[3]
    student_record['status'] = student_row[4]
    
    student_ref = db.collection('student').document(student_row[0])
    
    batch.set(student_ref, student_record)
    
    for takes_row in takes_rows:
        if student_row[0] == takes_row[0]:
            class_record = {}
            class_record['cno'] = takes_row[1]
            class_record['cname'] = takes_row[2]
            class_record['credits'] = takes_row[3]
            class_record['grade'] = takes_row[4]
            
            classes_ref = student_ref.collection('classes').document(takes_row[1])
        
            batch.set(classes_ref, class_record)
    
    batch.commit()

In [ ]:
batch = db.batch()

df = pd.read_csv('fs_college/instructor.csv', sep=',', header=0, lineterminator='\n')
instructor_rows = df.values.tolist()

df = pd.read_csv('fs_college/teaches.csv', sep=',', header=0, lineterminator='\n')
teaches_rows = df.values.tolist()

for instructor_row in instructor_rows:
    
    instructor_record = {}
    instructor_record['tid'] = instructor_row[0]
    instructor_record['instructor_name'] = instructor_row[1]
    instructor_record['dept'] = instructor_row[2]
    
    instructor_ref = db.collection('instructor').document(instructor_row[0])
    
    batch.set(instructor_ref, instructor_record)
    
    for teaches_row in teaches_rows:
        
        if instructor_row[0] == teaches_row[0]:
            
            class_record = {}
            class_record['cno'] = teaches_row[1]
            class_record['cname'] = teaches_row[2]
            class_record['credits'] = teaches_row[3]
            
            classes_ref = instructor_ref.collection('classes').document(teaches_row[1])
        
            batch.set(classes_ref, class_record)
    
    batch.commit()

#### 3. Run some queries

In [ ]:
student_ref = db.collection('student').document('paulg')
result = student_ref.get()

if result.exists:
    print(f'{result.id} => {result.to_dict()}')
else:
    print('No such student')

In [ ]:
student_ref = db.collection('student')
query = student_ref.where('status', '==', 'CUR')
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

In [ ]:
query = db.collection('instructor').document('mitra').collection('classes')
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

### Prompt 1: Get all classes which are worth 3 credits. Return the attributes of those classes. 

In [ ]:
query = db.collection_group('classes').where('credits', '==', 3)
results = query.stream()
for result in results:
    print(f'{result.id} => {result.to_dict()}')

### Prompt 2: What does the following query do? Write a short markdown description below the code

In [ ]:
instructor_ref = db.collection('instructor')
results = instructor_ref.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')
    sresults = instructor_ref.document(result.id).collection('classes').stream()
    
    for sresult in sresults:
        print(f'{sresult.id} => {sresult.to_dict()}')

#### For each instructor record, find the classes that they teach. Print the instructor's information followed by their classes.